Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.
SPDX-License-Identifier: Apache-2.0


## Using Hugging Face Pipelines on Intel&reg; Gaudi&reg; 2 - Image Classification
This section showcases how to use the Hugging Face Transformers pipeline API to run an image classification task on Intel Gaudi.

Hugging Face pipeline is an easy way to use models for inference. It is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks. We choose "image-classification" task here.    
Pipeline workflow is defined as a sequence of the following operations:

        Input -> Preprocessing -> Model Inference -> Post-Processing (Task dependent) -> Output
Pipeline supports running on CPU or GPU through the device argument. Users can specify device argument, for example, we set device="hpu" in this case. "adapt_transformers_to_gaudi" will replace some Transformers' methods for equivalent methods optimized for Intel Gaudi.

### The First step is Install the Hugging Face Optimum Habana Library

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Hugging_Face_pipelines
!pip install optimum-habana==1.15.0

### Import all neccessary dependencies and call adapt_transformers_to_gaudi()

In [ ]:
import PIL.Image
import requests
import torch
from transformers import pipeline

from optimum.habana.transformers.modeling_utils import adapt_transformers_to_gaudi
from habana_frameworks.torch.hpu import wrap_in_hpu_graph

The command below may be needed to modify the existing Hugging Face model classes to use the Intel Gaudi specific version of the model classes.

In [3]:
adapt_transformers_to_gaudi()

### Download and initialize the image

In [4]:
image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
image = PIL.Image.open(requests.get(image_url, stream=True, timeout=3000).raw).convert("RGB")

In [ ]:
# display image
image

### Setup the pipeline
To setup the Hugging Face pipeline we set the following  
1. choose the Hugging Face task: for this, we use **"image-classification"**
2. Set the device to **"hpu"** which allows the pipeline to run on Intel Gaudi
3. Choose model **"google/vit-base-patch32-384"** 

Finally we'll use the "wrap_in_hpu_graph" to wrap the module forward function with HPU Graphs. This wrapper captures, caches and replays the graph. More info [here](https://docs.habana.ai/en/latest/PyTorch/Inference_on_PyTorch/Inference_Using_HPU_Graphs.html).  

You will see that the Intel Gaudi will build the pipeline.

In [ ]:
classifier = pipeline("image-classification", model="google/vit-base-patch32-384", device="hpu")
classifier.model = wrap_in_hpu_graph(classifier.model)

### Execute the Pipeline and Display the results

The following results in a multi-class classification prediction:

In [ ]:
result = classifier(images=image)
print(result)

In [ ]:
exit()